In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import re
import math

import torchvision
from IPython.display import Image, display
from PIL import Image

import onnx

to_pil = torchvision.transforms.ToPILImage()
to_tensor = torchvision.transforms.ToTensor()

image = torchvision.io.read_image("/Users/khoi.ho/Downloads/110834364_p0.jpg",torchvision.io.ImageReadMode.RGB) / 256
image2 = Image.open("/Users/khoi.ho/Downloads/Screenshot 2023-08-21 14-04-25.png").convert("RGB")
image2 = to_tensor(image2)

class CReLU(nn.Module):
    def __init__(self):
        super(CReLU, self).__init__()
    def forward(self, x):
        return torch.cat((F.relu(x), F.relu(-x)), 1)


/usr/local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class Anime4kRestore(nn.Module):
    def __init__(self, block_depth=8, block_stack=5, channel=12):
        super(Anime4kRestore, self).__init__()
        self.conv_head = nn.Conv2d(3, channel, kernel_size=3, padding=1)
        self.conv_mid = nn.ModuleList([nn.Conv2d(
            channel*2, channel, kernel_size=3, padding=1) for _ in range(block_depth-1)])
        if block_stack != 1:
            self.conv_tail = nn.Conv2d(
                2*channel*block_stack, 3, kernel_size=1, padding=0)
        else:
            self.conv_tail = nn.Conv2d(2*channel, 3, kernel_size=3, padding=1)
        self.crelu = CReLU()
        self.block_no_stack = block_depth - block_stack

    def forward(self, x):
        out = self.crelu(self.conv_head(x))
        print(self.conv_head.weight.shape)
        if self.block_no_stack == 0:
            depth_list = [out]
            print(0, "----")
        else:
            depth_list = []
            print(0, "a")
        for i, conv in enumerate(self.conv_mid):
            out = self.crelu(conv(out))
            print(conv.weight.shape)
            if i >= self.block_no_stack - 1:
                depth_list.append(out)
                print(i+1, "----")
            else:
                print(i+1, "a")
        out = self.conv_tail(torch.cat(depth_list, 1))
        print(self.conv_tail.weight.shape)
        print("out")
        return torch.clamp(out + x, max=1.0, min=0.0)

    def import_param(self, filename):
        for param in self.parameters():
            param.requires_grad = False
        with open(filename) as f:
            text = f.read()
        pattern = r'-?\d+(\.\d{2,})(e-?\d+)?'
        iter = re.finditer(pattern, text)
        convert(self.conv_head, iter)
        for conv in self.conv_mid:
            convert(conv, iter)
        convert(self.conv_tail, iter)
        check = next(iter, None)
        if check == None:
            print("pass")
        else:
            print("---failed---\n", check)


def convert(c, iter):
    out_chan, in_chan, width, height = c.weight.shape
    # print(c.weight.shape)
    for to in range(math.ceil(out_chan/4)):
        for ti in range(math.ceil(in_chan/4)):
            for w in range(width):
                for h in range(height):
                    for i in range(min(4, in_chan)):
                        for o in range(min(4, out_chan)):
                            c.weight.data[to*4+o, ti*4+i, w,
                                          h] = float(next(iter).group(0))
        for i in range(min(4, out_chan)):
            c.bias.data[to*4+i] = float(next(iter).group(0))


model = Anime4kRestore(8, 5, 12)
model.import_param("tmp/Anime4K_Restore_CNN_UL.glsl")
# model = Anime4kRestore(8, 7, 8)
# model.import_param("tmp/Anime4K_Restore_CNN_VL.glsl")
# model = Anime4kRestore(4, 1, 8)
# model.import_param("tmp/Anime4K_Restore_CNN_L.glsl")
# model = Anime4kRestore(7, 7, 4)
# model.import_param("tmp/Anime4K_Restore_CNN_M.glsl")
# model = Anime4kRestore(3, 1, 4)
# model.import_param("tmp/Anime4K_Restore_CNN_S.glsl")

image2 = Image.open(
    "/Users/khoi.ho/Downloads/Screenshot 2023-08-21 14-04-25.png").convert("RGB")
image2 = to_tensor(image2)
# out = model(image2)
out = model(image2.unsqueeze(0))[0]
# display(to_pil(out))

pass
torch.Size([12, 3, 3, 3])
0 a
torch.Size([12, 24, 3, 3])
1 a
torch.Size([12, 24, 3, 3])
2 a
torch.Size([12, 24, 3, 3])
3 ----
torch.Size([12, 24, 3, 3])
4 ----
torch.Size([12, 24, 3, 3])
5 ----
torch.Size([12, 24, 3, 3])
6 ----
torch.Size([12, 24, 3, 3])
7 ----
torch.Size([3, 120, 1, 1])
out


In [ ]:
class Anime4kUpscale(nn.Module):
    def __init__(self, block_depth=8, block_stack=5, channel=12):
        super(Anime4kUpscale, self).__init__()
        self.conv_head = nn.Conv2d(3, channel, kernel_size=3, padding=1)
        self.conv_mid = nn.ModuleList([nn.Conv2d(
            channel*2, channel, kernel_size=3, padding=1) for _ in range(block_depth-1)])
        if block_stack != 1:
            self.conv_tail = nn.Conv2d(
                2*channel*block_stack, 12, kernel_size=1, padding=0)
        else:
            self.conv_tail = nn.Conv2d(2*channel, 12, kernel_size=3, padding=1)
        self.crelu = CReLU()
        self.block_no_stack = block_depth - block_stack
        self.ps = nn.PixelShuffle(2)

    def forward(self, x):
        out = self.crelu(self.conv_head(x))
        print(self.conv_head.weight.shape)
        if self.block_no_stack == 0:
            depth_list = [out]
            print(0, "----")
        else:
            depth_list = []
            print(0, "a")
        for i, conv in enumerate(self.conv_mid):
            out = self.crelu(conv(out))
            print(conv.weight.shape)
            if i >= self.block_no_stack - 1:
                depth_list.append(out)
                print(i+1, "----")
            else:
                print(i+1, "a")
        out = self.conv_tail(torch.cat(depth_list, 1))
        print(self.conv_tail.weight.shape)
        print("out")
        out = self.ps(out) + F.interpolate(x, scale_factor=2, mode='bilinear')
        return torch.clamp(out, max=1.0, min=0.0)

    def import_param(self, filename):
        for param in self.parameters():
            param.requires_grad = False
        with open(filename) as f:
            text = f.read()
        pattern = r'-?\d+(\.\d{4,})(e-?\d+)?'
        iter = re.finditer(pattern, text)
        convert(self.conv_head, iter)
        for conv in self.conv_mid:
            convert(conv, iter)
        convert(self.conv_tail, iter, True)
        check = next(iter, None)
        if check == None:
            print("pass")
        else:
            print("---failed---\n", check)
        
def convert(c, iter, doswap=False):
    swap = [0,2,1,3]
    out_chan, in_chan, width, height = c.weight.shape
    for to in range(math.ceil(out_chan/4)):
        for ti in range(math.ceil(in_chan/4)):
            for w in range(width):
                for h in range(height):
                    for i in range(min(4, in_chan)):
                        for o in range(min(4, out_chan)):
                            o = swap[o] if doswap else o
                            c.weight.data[to*4+o, ti*4+i, w, h] = float(next(iter).group(0))
        for o in range(min(4, out_chan)):
            o = swap[o] if doswap else o
            c.bias.data[to*4+o] = float(next(iter).group(0))


model = Anime4kUpscale(7, 5, 12)
model.import_param("tmp/Anime4K_Upscale_CNN_x2_UL.glsl")
# model = Anime4kUpscale(7, 7, 8)
# model.import_param("tmp/Anime4K_Upscale_CNN_x2_VL.glsl")
# model = Anime4kUpscale(3, 1, 8)
# model.import_param("tmp/Anime4K_Upscale_CNN_x2_L.glsl")

image2 = Image.open(
    "/Users/khoi.ho/Downloads/Screenshot 2023-08-21 14-04-25.png").convert("RGB")
image2 = to_tensor(image2)
out = model(image2.unsqueeze(0))[0]
# display(to_pil(out))

In [3]:
dummy_input = torch.randn(2, 3, 720, 1280)

# Export the model to ONNX format
onnx_path = "your_model.onnx"
torch.onnx.export(model, dummy_input, onnx_path)
# torch.onnx.export(model, dummy_input, onnx_path, verbose=True)




torch.Size([12, 3, 3, 3])
0 a
torch.Size([12, 24, 3, 3])
1 a
torch.Size([12, 24, 3, 3])
2 a
torch.Size([12, 24, 3, 3])
3 ----
torch.Size([12, 24, 3, 3])
4 ----
torch.Size([12, 24, 3, 3])
5 ----
torch.Size([12, 24, 3, 3])
6 ----
torch.Size([12, 24, 3, 3])
7 ----
torch.Size([3, 120, 1, 1])
out
